In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn import datasets
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

C:\Users\dhrub\AppData\Local\Temp\ipykernel_12284\2387614361.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
# Load the dataset
df = pd.read_csv('C:/Personal_Projects/projects/Credit Card Fraud Detection/credit_card_fraud_detection/data/raw/creditcard.csv')

In [4]:
df

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [6]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
df.shape

(284807, 31)

In [8]:
new_df = df.drop_duplicates()

In [9]:
new_df.shape

(283726, 31)

In [10]:
x = new_df.drop(columns=["Class","Time"])
y = new_df["Class"]

In [11]:
x.shape

(283726, 29)

In [12]:
y.shape

(283726,)

In [13]:
# Balancing with SMOTE
oversample = SMOTE()
X_resampled, y_resampled = oversample.fit_resample(x,y)
# X, y = oversample.fit_resample(df.drop('Class', axis=1), df['Class'])

In [14]:
# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=0)


In [15]:
# Feature Scaling
scaler = StandardScaler()  
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [16]:
# Model Building 
model = RandomForestClassifier()


In [17]:
# Model Training
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
# Prediction
y_pred = model.predict(X_test)

In [ ]:
# Model Evaluation
accuracy_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

In [ ]:

params = {
    'n_estimators': [100, 200, 500],
    'max_depth': [10, 50, 100], 
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 2, 5]
}

grid = GridSearchCV(RandomForestClassifier(), param_grid=params, cv=5)


In [ ]:
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
# Model Improvement 

# Try XGBoost classifier
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print(xgb.score(X_test, y_test))

# SMOTE handling class imbalance
from imblearn.over_sampling import SMOTE 
oversample = SMOTE()
X_sm, y_sm = oversample.fit_resample(X, y)

# Feature Selection
from sklearn.feature_selection import RFECV
selector = RFECV(RandomForestClassifier())
selector.fit(X_sm, y_sm)
X_reduced = selector.transform(X_sm)

# Stacked Model
from sklearn.ensemble import StackingClassifier
rf = RandomForestClassifier()
xgb = XGBClassifier()
stack = StackingClassifier(estimators=[('rf', rf), ('xgb', xgb)])
stack.fit(X_reduced, y_sm)

In [ ]:
# Model Artifact
import pickle
pickle.dump(model, open('rf_model.pkl', 'wb')) 

# Flask App
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route('/predict', methods=['POST'])  
def predict():
    data = request.get_json() 
    # preprocessing
    
    prediction = model.predict(data) 
    return jsonify({'prediction': prediction})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=9696) 

# Dockerfile
FROM python:3.7
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
EXPOSE 9696
ENTRYPOINT ["python", "app.py"]

# Kubernetes Deployment
apiVersion: apps/v1
kind: Deployment
metadata:
  name: fraud-deployment
spec:
  replicas: 3
  selector:
    matchLabels:
      app: fraud
  template:
    metadata:
      labels:
        app: fraud
    spec:
      containers:
      - name: fraud-model
        image: fraud_image
        ports:
        - containerPort: 9696
        
# Horizontal Pod Autoscaler 
apiVersion: autoscaling/v1
kind: HorizontalPodAutoscaler
metadata:
  name: fraud-hpa
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: fraud-deployment
  minReplicas: 3
  maxReplicas: 10
  targetCPUUtilizationPercentage: 50